In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/datasets/

/content/drive/My Drive/datasets


In [3]:
!ls

 beautfulsoup.py    DataScience			  news.train
 cc.si.300.bin	   'fasttext_trained model.bin'   stem_dictonary.txt
 cc.si.300.vec	    file_name.csv		  stopwords.txt
 cleaned_data.csv   file.txt			  sufix_list.txt
 data.csv	    news.test


In [4]:
import numpy as np
import pandas as pd
from plotly.offline import iplot
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
df = pd.read_csv('data.csv')
data=df.dropna()
data.columns =['News', 'Label'] # add labels

In [6]:
data['Label'].value_counts()

3    12450
2     9960
1     9959
Name: Label, dtype: int64

#Basic NLP

In [7]:
with open('stopwords.txt', 'r') as f:
    sinhala_stopwords = [line.strip() for line in f]
with open('sufix_list.txt', 'r') as f:
    suffix_list = [line.strip() for line in f]

In [8]:
#cleaning
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(sinhala_stopwords)
SUFFIX = set(suffix_list)

def clean_text(text):
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    #text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('\\', '')
    text = text.replace('-', '')
    text = text.replace('\'', '')
    text = text.replace('.', '')
    
    text = ''.join([i for i in text if not i.isdigit()])
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    text = ' '.join(word for word in text.split() if word not in SUFFIX)
    return text

data['News'] = data['News'].apply(clean_text)

In [9]:
with open('stem_dictonary.txt') as f:
    lines = [line.strip() for line in f]
stem_list=[]
for i in lines:
  x = i.split("\t")
  stem_list.append(x)

def stemming(string1):
  wordlist=word_tokenize(string1)
  for i in range(len(wordlist)):
    for j in stem_list:
      if j[0]==wordlist[i]:
        wordlist[i]=j[1]

  return ' '.join(wordlist)

In [10]:
def reEncode(text):
  tokens = nltk.word_tokenize(text)
  regex = re.compile(u'[^\u0D80-\u0DFF]', re.UNICODE)
  tokens = [regex.sub('', w) for w in tokens]
  return ' '.join(tokens)

In [11]:
data['News'] = data.apply(lambda x: stemming(x['News']), axis=1)

In [12]:
data['News'] = data.apply(lambda x: reEncode(x['News']), axis=1)

In [13]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=25)

In [14]:
import numpy as np, pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
sns.set() # use seaborn plotting style

In [15]:
# Build the model
model = make_pipeline(TfidfVectorizer(), MultinomialNB())
# Train the model using the training data
model.fit(train_data.News, train_data.Label)
# Predict the categories of the test data
predicted_categories = model.predict(test_data.News)

In [16]:
TestNews=list(data['News'])

In [17]:
for ind in range(len(TestNews)):
    print(TestNews[ind], predicted_categories[ind])
    if(ind==10):
      break

දකුණු අප්රිකා ඉන්දියා දෙවන තරඟ අද 2
මල්ෂා ෂෙහානි වාර්තාව සමගි ලංකා ජය ගෙනෙයි 1
ලංකා කාන්තා පිල මැලේසියා පරදයි 3
සමස්ත ලංකා පාසල් ක්රීඩා තරග යළි 2
ලංකා කාන්තා ක්රිකට් කණ්ඩාය තායිලන්ත පරදයි 3
ලංකා කාන්තා කණ්ඩායමට ලකුණු පරාජයක් 1
ප්රවීනයන්ගේ අවසන් තරඟ රාත්රී 3
බාබර් කෝලි සම කරයි 3
දකුණු අප්රිකාව පරදා ඉන්දියාව කඩුලු ජය ගනියි 1
ලෝක කුසලානයට කණ්ඩායමේ මානසිකත්ව නවීඩ් නවාස්ගෙන් තක්සේරුවක 2
මහවැලි කලාපවල සංස්කෘතික අධ්යාපනික ක්රීඩා පුහුණු සංචිත පිහිටුවයි 2


In [18]:
all_predictions = model.predict(data.News)
confusion_matrix(data['Label'], all_predictions)
print("The accuracy is {}".format(accuracy_score(data.Label, all_predictions)))

The accuracy is 0.781519354938367


In [19]:
from sklearn import metrics
print(metrics.confusion_matrix(data['Label'], all_predictions))

[[ 8150   469  1340]
 [  620  7052  2288]
 [  930  1425 10095]]


In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
classifier = make_pipeline(TfidfVectorizer(), LogisticRegression(max_iter=300))
classifier.fit(train_data.News,train_data.Label)
from sklearn import metrics
predicted = classifier.predict(test_data.News)
print("Logistic Regression Accuracy:",metrics.accuracy_score(test_data.Label, predicted))


Logistic Regression Accuracy: 0.7703120172999691
